In [1]:
import random

#Pennylane
import pennylane as qml
from pennylane import numpy as np

#PyTorch
import torch
import torchvision
from torchvision import datasets, transforms

In [2]:
dev = qml.device('default.qubit', wires = 6)
n_qubits = 6
n_layer_steps = 5
n_layers_to_add = 2
dim = 2**(n_qubits)

In [3]:
def set_random_gates(n_qubits: int):
    
    gate_set = [qml.RX, qml.RY, qml.RZ]
    chosen_gates = []
    for i in range(n_qubits):
        chosen_gate = random.choice(gate_set)
        chosen_gates.append(chosen_gate)
    return chosen_gates

def norm(complex_vector: np.array) -> float:
    """Returns the norm of a complex vector.
    
    Args:
       complex_vector (np.arrray of shape (dim,)):
           complex vector with an arbitrary dimension.
           
       
    Returns:
        norm (float): norm or magnitude of the complex vector.
            
    """
    
    #define the norm of a complex vector here
    norm = np.sqrt(sum([np.square(np.absolute(complex_vector[i])) for i in range(complex_vector.shape[0])]))
    
    return norm

def random_quantum_state(dim: int, amplitude_type: str = 'complex') -> np.array:
    """Creates a normalized random real or complex vector of a defined
    dimension.
    
    Args:
        dim (int): integer number specifying the dimension
            of the vector that we want to generate.
            
        amplitude_type (str): string indicating if you want to create a
            vector with 'complex' or 'real' number amplitudes.
            
    Returns:
        (np.array): normalized real or complex vector of the given
            dimension.
    """
    #generate an unnormalized complex vector of dimension = dim
    if amplitude_type == 'complex':
        Z = np.random.random(dim) + np.random.random(dim)*1j
    
    #generate an unnormalized real vector of dimension = dim
    elif amplitude_type == 'real':
        Z = np.random.random(dim)
    
    #normalize the complex vector Z
    Z /= norm(Z)
    
    return Z

def density_matrix(target_vector: np.array) -> np.array:
    """Return the density matrix of a pure state.
    
    Arguments:
        target_vector (np.array): Complex vector array.
        
    Returns:
        (np.array): Density matrix of shape (dim, dim),
            where dim is the dimension of the target_vector.
    """
    target_vector_reshape = target_vector.reshape((target_vector.shape[0],1))
    density_matrix = target_vector_reshape @ np.transpose(np.conjugate(target_vector_reshape))
    
    return density_matrix

In [4]:
target_vector = random_quantum_state(dim)
density_matrix = density_matrix(target_vector)

### Templates for Phase I

In [5]:
@qml.template
def apply_layer(gates, weights):
    
    for i in range(n_qubits):
        gates[i](weights[i], wires = i)
    
    tuples = [(i,i+1) for i in range(n_qubits-1)]

    for tup in tuples:
        qml.CZ(wires=[tup[0], tup[1]])
        
@qml.template #template for non-trainable part of the quantum circuit
def frozen_layers(frozen_layer_gates, frozen_layer_weights): 

    for i in range(len(frozen_layer_gates)):
        apply_layer(frozen_layer_gates[i], frozen_layer_weights[i])

@qml.qnode(dev)
def trainable_circuit(params):
    
    #apply frozen layers
    frozen_layers(layer_gates, layer_weights)
    
    #apply trainable layers
    new_weights = []
    for i in range(n_layers_to_add):
        new_weights.append(params[int(i*n_qubits):int((i+1)*n_qubits)])

    for i in range(n_layers_to_add):
        apply_layer(new_gates[i], new_weights[i])
        
    wirelist = [i for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix, wirelist))

In [6]:
def cost(x):
    
    return (1.0 - trainable_circuit(x))

In [7]:
layer_gates = []
layer_weights = []
n_layers = 2
for i in range(n_layers):
    layer_gates.append(set_random_gates(n_qubits))
    layer_weights.append(np.random.rand(n_qubits)*2*np.pi)
    
new_gates = [set_random_gates(n_qubits) for i in range(n_layers_to_add)]
params = np.random.rand(n_qubits*n_layers_to_add)*2*np.pi

trainable_circuit(params)

0.0003779532166912125

In [8]:
layer_gates = []
layer_weights = []
cost_list = []
for step in range(n_layer_steps):
    
    new_gates = [set_random_gates(n_qubits) for i in range(n_layers_to_add)]
    init_params = np.zeros(n_qubits*n_layers_to_add)
    
    opt = qml.GradientDescentOptimizer(stepsize = 0.4)

    opt_steps = 100

    params = init_params
    
    for i in range(opt_steps):

        params = opt.step(cost, params)
    
    
    print(step)
    print(layer_weights)
    print(cost(params))
    cost_list.append(cost(params))
    
    trained_weights = [params[int(i*n_qubits):int((i+1)*n_qubits)] for i in range(n_layers_to_add)]
    
    layer_gates += new_gates
    layer_weights += trained_weights

0
[]
0.9803189175700198
1
[array([-1.88500289e-02, -7.63278329e-18,  2.92986693e-01, -4.28631584e-02,
        4.25800949e-02, -2.03202408e-01]), array([-0.01885003,  0.11791   ,  0.07449777,  0.42752853,  0.03339696,
       -0.08276643])]
0.7594185979414175
2
[array([-1.88500289e-02, -7.63278329e-18,  2.92986693e-01, -4.28631584e-02,
        4.25800949e-02, -2.03202408e-01]), array([-0.01885003,  0.11791   ,  0.07449777,  0.42752853,  0.03339696,
       -0.08276643]), array([ 1.21485553,  1.65020104, -1.24182455,  0.95286084,  0.02694313,
       -0.5171402 ]), array([ 0.11813642,  0.10972631, -0.12449923, -0.76713037,  0.02694313,
       -0.5171402 ])]
0.5230370890092713
3
[array([-1.88500289e-02, -7.63278329e-18,  2.92986693e-01, -4.28631584e-02,
        4.25800949e-02, -2.03202408e-01]), array([-0.01885003,  0.11791   ,  0.07449777,  0.42752853,  0.03339696,
       -0.08276643]), array([ 1.21485553,  1.65020104, -1.24182455,  0.95286084,  0.02694313,
       -0.5171402 ]), array([ 0.1

In [9]:
layer_weights

[array([-1.88500289e-02, -7.63278329e-18,  2.92986693e-01, -4.28631584e-02,
         4.25800949e-02, -2.03202408e-01]),
 array([-0.01885003,  0.11791   ,  0.07449777,  0.42752853,  0.03339696,
        -0.08276643]),
 array([ 1.21485553,  1.65020104, -1.24182455,  0.95286084,  0.02694313,
        -0.5171402 ]),
 array([ 0.11813642,  0.10972631, -0.12449923, -0.76713037,  0.02694313,
        -0.5171402 ]),
 array([ 0.25267196, -0.02662812,  0.39235312,  0.38444078,  0.63439665,
         1.11155113]),
 array([-0.01434425,  0.15313149, -0.39482855,  0.02251025, -0.12708225,
         0.31694366]),
 array([ 0.01075021, -0.01211775, -0.19662557, -0.09183198,  1.34770013,
        -0.00946216]),
 array([ 0.05425855,  0.03336504,  0.26009126,  0.21799258,  0.50105599,
        -0.00946216]),
 array([0.1354826 , 0.01763158, 0.36127584, 0.10787329, 0.00344715,
        0.03953567]),
 array([ 0.00325754, -0.00696433, -0.12513489,  0.04353759,  0.05932017,
        -0.00755341])]

### Phase II

In [10]:
partition_percentage = 0.5
partition_size = int(n_layer_steps*n_layers_to_add*partition_percentage)
n_partition_weights = partition_size*n_qubits
n_sweeps = 2

In [11]:
print(layer_weights[:5])
print(n_partition_weights)
print(partition_size)

[array([-1.88500289e-02, -7.63278329e-18,  2.92986693e-01, -4.28631584e-02,
        4.25800949e-02, -2.03202408e-01]), array([-0.01885003,  0.11791   ,  0.07449777,  0.42752853,  0.03339696,
       -0.08276643]), array([ 1.21485553,  1.65020104, -1.24182455,  0.95286084,  0.02694313,
       -0.5171402 ]), array([ 0.11813642,  0.10972631, -0.12449923, -0.76713037,  0.02694313,
       -0.5171402 ]), array([ 0.25267196, -0.02662812,  0.39235312,  0.38444078,  0.63439665,
        1.11155113])]
30
5


In [12]:
#trainable_layers = layer_gates[:partition_size]
#trainable_weights = layer_weights[:partition_size]
#weights_flatten = np.concatenate(trainable_weights).ravel()

#is going to depend on the part of the circuit
@qml.qnode(dev)
def trainable_partition(params):
    
    if partition == 1:
        #apply trainable partition
        weights = []
        for i in range(partition_size):
            weights.append(params[int(i*n_qubits):int((i+1)*n_qubits)])
        
        for i in range(len(layer_gates[:partition_size])):
            apply_layer(layer_gates[:partition_size][i], weights[i])
            
        #apply non-trainable partition
        for i in range(len(layer_gates[partition_size:])):
            apply_layer(layer_gates[partition_size:][i], layer_weights[partition_size:][i])
        
    elif partition == 2:
        #apply non-trainable partition
        for i in range(len(layer_gates[:partition_size])):
            apply_layer(layer_gates[:partition_size][i], layer_weights[:partition_size][i])
    
        #apply trainable partition
        weights = []
        for i in range(partition_size):
            weights.append(params[int(i*n_qubits):int((i+1)*n_qubits)])
        
        for i in range(len(layer_gates[partition_size:])):
            apply_layer(layer_gates[partition_size:][i], weights[i])
            
    wirelist = [i for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix, wirelist))

In [13]:
def cost(x):
    
    return (1.0 - trainable_partition(x))

In [14]:
for sweep in range(n_sweeps):
    
    partition = 1
    trainable_weights = layer_weights[:partition_size]

    init_params = np.concatenate(trainable_weights).ravel()
    
    opt = qml.GradientDescentOptimizer(stepsize = 0.4)

    opt_steps = 100

    params = init_params
    
    for i in range(opt_steps):

        params = opt.step(cost, params)
        
    trained_weights = [params[int(i*n_qubits):int((i+1)*n_qubits)] for i in range(partition_size)]
    
    layer_weights[:partition_size] = trained_weights
    
    print(cost(params))

    partition = 2
    trainable_weights = layer_weights[partition_size:]

    init_params = np.concatenate(trainable_weights).ravel()
    
    opt = qml.GradientDescentOptimizer(stepsize = 0.4)

    opt_steps = 100

    params = init_params
    
    for i in range(opt_steps):

        params = opt.step(cost, params)
        
    trained_weights = [params[int(i*n_qubits):int((i+1)*n_qubits)] for i in range(partition_size)]
    
    layer_weights[partition_size:] = trained_weights
    
    print(cost(params))

0.19341886341920778
0.1859906609377724
0.18510864040715902
0.18364257037953735


### Complete-depth learning

In [15]:
@qml.qnode(dev)
def complete_depth_learning(params):
    
    weights = [params[int(i*n_qubits):int((i+1)*n_qubits)] for i in range(len(layer_gates))]
    
    for i in range(len(layer_gates)):
        apply_layer(layer_gates[i], weights[i])
        
    wirelist = [i for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix, wirelist))

In [16]:
def cost(x):
    
    return (1.0 - complete_depth_learning(x))

In [17]:
init_params = np.zeros(n_qubits*len(layer_gates))
    
opt = qml.GradientDescentOptimizer(stepsize = 0.4)

opt_steps = 200

params = init_params

for i in range(opt_steps):

    params = opt.step(cost, params)

print(cost(params))

0.1860668415207497
